In [1]:
# http://www.kemik.yildiz.edu.tr/?id=28

import numpy as np
import os

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

Using TensorFlow backend.


In [4]:
  lines = []

# Her turlu kucuk karaktere cevirdigimiz icin, Ü, Ş 
# gibi karakterlere u, s vs. de atayabilirdik.
replace_table = {
    b'\xfd' : b'\x69', # i
    b'\xe7' : b'\x63', # c
    b'\xfe' : b'\x73', # s
    b'\xf0' : b'\x67', # g
    b'\xf6' : b'\x6f', # o
    b'\xfc' : b'\x75', # u
    b'\xc7' : b'\x43', # Ç -> C
    b'\xdd' : b'\x49', # İ -> I
    b'\xde' : b'\x53', # Ş -> S
    b'\xd6' : b'\x4f', # Ö -> O
    b'\xdc' : b'\x55', # Ü -> U    
}
base_dir = "raw_texts/"
count = 0

for d in os.listdir(base_dir):
    for w in os.listdir(base_dir + d + '/'):
        w_dir = base_dir + d + '/' + w
        
        with open(w_dir, 'rb') as f:
            for line in f:
                for k, v in replace_table.items():
                    line = line.replace(k, v)
                line = line.strip().lower()
                line = line.decode('utf-8', 'ignore')
                if len(line) == 0:
                    continue
                lines.append(line)
        count += 1

print(count)

270


In [0]:
text = " ".join(lines)

In [6]:
chars = set(text)
print(chars)

nb_chars = len(chars)
print(nb_chars)

char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

{'j', '"', 'd', '*', 'e', 'a', ':', ';', '=', '3', ',', '(', 'z', '?', 'r', '/', "'", '<', 'h', '-', '6', 'v', 'o', '4', 'u', 'q', 'b', ')', '1', 'k', '2', 'l', 'f', '7', 'n', '0', '+', 'y', 'm', 'c', ' ', 'x', 'i', '.', 's', '8', 'w', '%', '9', '5', '!', 'p', 'g', 't'}
54


In [0]:
STEP = 2
SEQLEN = 15

input_chars = []
labels = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i+SEQLEN])
    labels.append(text[i+SEQLEN])

In [18]:
input_chars[:10]

["ozal'i cevresi ",
 "al'i cevresi ol",
 "'i cevresi oldu",
 ' cevresi oldurd',
 'evresi oldurdu!',
 'resi oldurdu! s',
 'si oldurdu! sek',
 ' oldurdu! sekiz',
 'ldurdu! sekizin',
 'urdu! sekizinci']

In [19]:
labels[:10]

['o', 'd', 'r', 'u', ' ', 'e', 'i', 'i', 'c', ' ']

In [20]:
# One-hot encode the characters 
len_of_input = len(input_chars)
print(len_of_input)

x = np.zeros((len_of_input, SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len_of_input, nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        x[i, j, char2index[ch]] = 1
    y[i, char2index[labels[i]] ] = 1

465346


In [21]:
model = Sequential()
model.add(SimpleRNN(128, input_shape=(SEQLEN, nb_chars), unroll=True))
model.add(Dense(nb_chars, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 128)               23424     
_________________________________________________________________
dense_4 (Dense)              (None, 54)                6966      
Total params: 30,390
Trainable params: 30,390
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds, temp=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temp
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  preds = preds.ravel()
  probas = np.random.multinomial(1, preds, 1, )
  
  return np.argmax(probas)

In [34]:
for iteration in range(30):
    print("=" * 50)
    print("Iteration #: ", iteration)
    
    model.fit(x, y, batch_size=128, epochs=1)
    
    index = np.random.randint(0, len_of_input - SEQLEN - 1)
    test_chars = text[index: index + SEQLEN] 
    
    for i in range(200):
        test_x = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            test_x[0, i, char2index[ch]] = 1
        
        preds = model.predict(test_x, verbose=0)
        index = sample(preds, 0.5)
        ypred = index2char[index]
        print(ypred, end='')
        test_chars = test_chars[1:] + ypred
    print()

Iteration #:  0
Epoch 1/1
454016/465346 [============================>.] - ETA: 0s - loss: 1.9810

465346/465346 [==============================] - 38s 81us/step - loss: 1.9806
i yapaclari da yaralari hali tat yazeleren sir de oldugu bir terkiye ?unu ve basini bayatin, hasan donemlerin ecine karaflarinin karalarini istasa caksir esti. ayladini aliyoruz. bun konum" deyeklerin
Iteration #:  1
Epoch 1/1
149632/465346 [========>.....................] - ETA: 25s - loss: 1.9288

465346/465346 [==============================] - 38s 81us/step - loss: 1.9131
turk siyasan devret eli gatir basari alkinda yani sayisinda bulumda, yakanat varilanin onemini tarsin sonra silan olarak, bunun hali olucu karmalin dopran buyakalar isten yapma nekirlerin bir de yapal
Iteration #:  2
Epoch 1/1
 96000/465346 [=====>........................] - ETA: 30s - loss: 1.8709

465346/465346 [==============================] - 38s 82us/step - loss: 1.8642
r gelesi oluyor. gerekini alani cakaraktan daha saderi isiyorum de bir derlerin tem olabilir mu? degiliyor. siyasi istiki bile durdu. ecevit de degil mu.. hafta alasini gerek ankarani icin ismalarini 
Iteration #:  3
Epoch 1/1
 90112/465346 [====>.........................] - ETA: 30s - loss: 1.8328

465346/465346 [==============================] - 37s 80us/step - loss: 1.8292
ti kaldiri altini da kararlik... bu sorunun istanetin oylaya kapilan bu alabilirinin ettigi kolak rapatlarini degilimi altina karsilan buyuk yapmaz ciyasetirinini alayacak, bu alayasinin derekli olara
Iteration #:  4
Epoch 1/1
 88704/465346 [====>.........................] - ETA: 30s - loss: 1.7993

...

465346/465346 [==============================] - 38s 82us/step - loss: 1.6466
diye sormak gecmiyor. anna pararlar bir kalip bir onunda heyemenlerinin baskani de boglanin mesil ediliyor. piyasalarin de katilarla kapadilarini bir sermayen anlatmis. basarilarinda de bu sorumlarin 
Iteration #:  26
Epoch 1/1
 88448/465346 [====>.........................] - ETA: 30s - loss: 1.6332

465346/465346 [==============================] - 38s 81us/step - loss: 1.6444
nun gereken yarat genel tartisilara alinmasi icin dunya alman dahiliyle kararlarin baslarindan beni yani bir tekilleri gozunlerin etkiniyle sanali kurulusun isin gerekiyor. annan bir teror olmasini al
Iteration #:  27
Epoch 1/1
 88448/465346 [====>.........................] - ETA: 30s - loss: 1.6334

465346/465346 [==============================] - 38s 81us/step - loss: 1.6425
 karlarta degisimi sayion uzerine kurulmadi. kitapin gelismeye alman uneri bir siyasi karara goresi ve cikarak hastanesi ihrahit ule de bu durumuz. baska sermaye gecen arasindan plan sorumlu, derisind
Iteration #:  28
Epoch 1/1
 87040/465346 [====>.........................] - ETA: 30s - loss: 1.6294

465346/465346 [==============================] - 38s 81us/step - loss: 1.6393
adilar. bu konusma sadece islami aradik soylemedigi icin kendi simdi meclis istanbul alma geldigi ikili bir seytir" bir kezdir. tartisma murastin yilinda degil. sozune basi olursa karsilari bekleneri 
Iteration #:  29
Epoch 1/1
 86400/465346 [====>.........................] - ETA: 31s - loss: 1.6286

465346/465346 [==============================] - 38s 81us/step - loss: 1.6373
abderlik olgus siranin yazinda orkeleri malisin milletvekili ifadelerin idamis gerektigi" kivini yaristirdi. boyle bir almadik ve diyor. annede bir arafindan bankalar adim olmasini icin de kararim izl
